In [7]:
# import pyttsx3
# engine = pyttsx3.init() # object creation

# """ RATE"""
# engine.setProperty('rate', 125)     # setting up new voice rate
# rate = engine.getProperty('rate')   # getting details of current speaking rate
# print (rate)                        #printing current voice rate


# """VOLUME"""
# volume = engine.getProperty('volume')   #getting to know current volume level (min=0 and max=1)
# print (volume)                          #printing current volume level
# engine.setProperty('volume',1.0)    # setting up volume level  between 0 and 1

# """VOICE"""
# voices = engine.getProperty('voices')       #getting details of current voice
# #engine.setProperty('voice', voices[0].id)  #changing index, changes voices. o for male
# engine.setProperty('voice', voices[1].id)   #changing index, changes voices. 1 for female

# engine.say("Hello World!")
# engine.say('My current speaking rate is ' + str(rate))
# engine.runAndWait()
# engine.stop()

# """Saving Voice to a file"""
# # On linux make sure that 'espeak' and 'ffmpeg' are installed
# engine.save_to_file('Hello World', 'test.mp3')
# engine.runAndWait()

In [8]:
# import ebooklib
# from ebooklib import epub
# from bs4 import BeautifulSoup
# import warnings

# # Suppress specific warnings from ebooklib
# warnings.filterwarnings('ignore', category=UserWarning, module='ebooklib')
# warnings.filterwarnings('ignore', category=FutureWarning, module='ebooklib')

# def epub_to_text(epub_path, output_path):
#     book = epub.read_epub(epub_path)
#     with open(output_path, 'w', encoding='utf-8') as f:
#         for item in book.get_items_of_type(ebooklib.ITEM_DOCUMENT):
#             content = item.get_content()
#             soup = BeautifulSoup(content, 'html.parser')
#             f.write(soup.get_text())

# # Example usage
# epub_to_text(r"C:\Users\khang\Desktop\DS_Work\tts\Short stories\the dark forest.epub", r"C:\Users\khang\Desktop\DS_Work\tts\Short stories\The Dark Forest.txt")

In [12]:
import edge_tts
import pandas as pd  # Import pandas for DataFrame

async def fetch_voices():
    # Create an instance of VoicesManager to get available voices
    voices_manager = await edge_tts.VoicesManager.create()
    voices = voices_manager.voices

    # Store the voices in a list with the correctly formatted ShortName
    voices_list = []
    for voice in voices:
        # Extract the name from the 'Name' field, which is within parentheses
        name_parts = voice['Name'].split('(')
        if len(name_parts) > 1:
            # Extract the text inside the parentheses
            voice_short_name = name_parts[1].split(", ")[1].replace(")", "")
        else:
            # Fallback to the entire name if format doesn't match
            voice_short_name = voice['Name']
        
        # Construct the list with the formatted ShortName without the prefix
        voices_list.append([
            f"{voice['Locale']}-{voice_short_name}", 
            voice['Locale'], 
            voice['Gender']
        ])
    
    return voices_list

# Fetch the voices list
voices_list_output = await fetch_voices()

# Define headers for the DataFrame
headers = ["ShortName", "Locale", "Gender"]

# Create a DataFrame using pandas
Voice_Name_List = pd.DataFrame(voices_list_output, columns=headers)

# # Display the DataFrame
# print(df_voices)


In [15]:
import asyncio
import io
import edge_tts
import pygame

async def text_to_speech(text, voice):
    # Create an instance of the Communicate class with the desired text and voice
    communicate = edge_tts.Communicate(
        text=text,
        voice=voice
    )

    # Create an in-memory buffer to store the audio
    audio_buffer = io.BytesIO()

    # Stream the TTS result to the in-memory buffer
    async for chunk in communicate.stream():
        if chunk["type"] == "audio":
            audio_buffer.write(chunk["data"])

    # Reset buffer position to the beginning
    audio_buffer.seek(0)
    return audio_buffer

async def generate_audio_for_specific_voice(text):
    # Specify the voice you want to use. You can find a list generated from the code above.
    # Replace 
    specific_voice = "en-US-AvaMultilingualNeural"

    print(f"Generating audio for voice: {specific_voice}")
    audio_buffer = await text_to_speech(text, specific_voice)

    # Initialize pygame mixer
    pygame.mixer.init()

    # Load audio data from the buffer and play it
    try:
        # Create a pygame sound object from the in-memory buffer
        sound = pygame.mixer.Sound(file=audio_buffer)
        sound.play()

        # Wait until the sound has finished playing
        while pygame.mixer.get_busy():
            await asyncio.sleep(1)
    except Exception as e:
        print(f"Error playing audio: {e}")

# Specify the text you want to synthesize
text = "Hello world, I'm so happy to be here!"

# Run the generation for the specific voice
await generate_audio_for_specific_voice(text)


Generating audio for voice: en-US-AvaMultilingualNeural
